In [32]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", message="Could not infer format")


# Preprocessing: Nabahan Full Details Dataset

## Dataset Overview
This dataset contains detailed tender information scraped from the Etimad platform. It includes 2414 tenders with 41 columns covering:

- Tender identifiers (`tender_number`, `reference_number`)
- Titles and descriptions (`tender_name`, `tender_purpose`, `details`)
- Budget and financial fields (`tender_document_value`, `initial_guarantee_required`, `final_guarantee`)
- Status, type, and activity fields (`tender_status`, `tender_type`, `competition_activity`, `classification_field`)
- Dates (`submission_deadline`, `opening_date`, `evaluation_date`, `work_start_date`, etc.)
- Locations (`government_entity`, `execution_location`, `opening_location`)
- Suppliers (`suppliers_applied`, `suppliers_awarded`)
- Additional operational and administrative fields

Many columns contain missing values (`NaN`) due to incomplete data. Sparse columns are kept intact for future reference and potential analysis.

## Preprocessing Steps

1. **Rename Columns**
   - All columns were converted to **English snake_case** for consistency and ease of analysis.
   
2. **Normalize Budgets**
   - Values like `"مجانا"` (free) are converted to `0`.
   - Numeric values are converted to float, keeping missing or invalid values as `NaN`.
   
3. **Normalize Yes/No Fields**
   - Arabic `"نعم"` → `True`, `"لا"` → `False`.
   - Columns: `insurance_required`, `initial_guarantee_required`.
   
4. **Normalize Dates**
   - Converted all date fields to pandas datetime objects (`NaT` for missing/invalid values):
     `submission_deadline`, `opening_date`, `evaluation_date`, `work_start_date`, etc.
   - Warnings about unknown formats are expected and safe.
   
5. **Add Metadata**
   - Added `source_file` = `"nabahan_full_details.csv"`
   - Added `ingestion_date` = current date

6. **Keep All Columns**
   - All 41 columns were preserved, including sparse columns like `countries`, `maintenance_works`, and `suppliers_awarded`.

7. **Save Cleaned Dataset**
   - Cleaned data saved to: `processed/core/tenders_clean.csv`
   - Ready for merging with other datasets and for SQL/NLP agent usage.

---

> This preprocessing step ensures **all real data is preserved**, cleaned, and normalized, while keeping missing values intact. It forms the foundation for building the InsightAgent’s **unified tenders database**.


In [33]:
df = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\core\nabahan_full_details.csv")
df.head()

,الرابط,اسم المنافسة,رقم المنافسة,الرقم المرجعي,الغرض من المنافسة,قيمة وثائق المنافسة,حالة المنافسة,مدة العقد,هل التأمين من متطلبات المنافسة,نوع المنافسة,...,عنوان الضمان الإبتدائى,أعمال الإنشاء,الحزمة,نوع الاتفاقية,مدة الاتفاقية,عدد ايام استلام الاستفسارات,عدد أيام استلام العرض,قائمة الموردين المتقدمين,قائمة الموردين المرسى عليهم - ( ترسية كاملة ),الدول
0,https://tenders.etimad.sa/Tender/DetailsForVis...,SLEEVE MEDIUM SCD EXPRESS KNEE LENGTH UNIVERSAL,40048995,2.601390e+11,الحاجة الماسة للبند لمســتشفى...عرض المزيد...ا...,مجانا,معتمدة,90 يوم,لا,شراء مباشر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://tenders.etimad.sa/Tender/DetailsForVis...,SLEEVE LARGE SCD EXPRESS KNEE LENGTH UNIVERSAL,40048996,2.601390e+11,الحاجة الماسة للبند لمســتشفى...عرض المزيد...ا...,مجانا,معتمدة,90 يوم,لا,شراء مباشر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://tenders.etimad.sa/Tender/DetailsForVis...,"CYSTOTOME IRRIGATING FORMED 27G X 5/8""",40049095,2.601390e+11,الحاجة الماسة للبند لمســتشفى...عرض المزيد...ا...,مجانا,معتمدة,90 يوم,لا,شراء مباشر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://tenders.etimad.sa/Tender/DetailsForVis...,ACYCLOVIR 3% OPHTHALMIC OINTMENT,40049063,2.601390e+11,الحاجة الماسة للبند لمســتشفى...عرض المزيد...ا...,200.00,معتمدة,90 يوم,لا,شراء مباشر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://tenders.etimad.sa/Tender/DetailsForVis...,"BAG, PLASTIC, TRANSPARENT, FOR MEDICINE, SIZE:...",40049073,2.601390e+11,الحاجة الماسة للبند لمســتشفى...عرض المزيد...ا...,200.00,معتمدة,90 يوم,لا,شراء مباشر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2414 entries, 0 to 2413
Data columns (total 41 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   الرابط                                           2414 non-null   object 
 1   اسم المنافسة                                     2413 non-null   object 
 2   رقم المنافسة                                     2413 non-null   object 
 3   الرقم المرجعي                                    2413 non-null   float64
 4   الغرض من المنافسة                                2413 non-null   object 
 5   قيمة وثائق المنافسة                              2413 non-null   object 
 6   حالة المنافسة                                    2413 non-null   object 
 7   مدة العقد                                        2413 non-null   object 
 8   هل التأمين من متطلبات المنافسة                   2413 non-null   object 
 9   نوع المنافسة                  

In [35]:
print(df.columns)

Index(['الرابط', 'اسم المنافسة', 'رقم المنافسة', 'الرقم المرجعي',
       'الغرض من المنافسة', 'قيمة وثائق المنافسة', 'حالة المنافسة',
       'مدة العقد', 'هل التأمين من متطلبات المنافسة', 'نوع المنافسة',
       'الجهة الحكوميه', 'الوقت المتبقى', 'طريقة تقديم العروض',
       'مطلوب ضمان الإبتدائي', 'الضمان النهائي',
       'آخر موعد لإستلام الإستفسارات', 'آخر موعد لتقديم العروض',
       'تاريخ فتح العروض', 'تاريخ فحص العروض', 'فترة التوقف',
       'التاريخ المتوقع للترسية', 'تاريخ بدء الأعمال / الخدمات',
       'بداية إرسال الأسئلة و الاستفسارات', 'اقصى مدة للاجابة على الاستفسارات',
       'مكان فتح العرض', 'مجال التصنيف', 'مكان التنفيذ', 'التفاصيل',
       'نشاط المنافسة', 'تشمل المنافسة على بنود توريد',
       'أعمال الصيانة والتشغيل', 'عنوان الضمان الإبتدائى', 'أعمال  الإنشاء',
       'الحزمة', 'نوع الاتفاقية', 'مدة الاتفاقية',
       'عدد ايام استلام الاستفسارات', 'عدد أيام استلام العرض',
       'قائمة الموردين المتقدمين',
       'قائمة الموردين المرسى عليهم  -  ( ترسية كاملة )', 'ا

In [36]:
df.rename(columns={
    'الرابط': 'url',
    'اسم المنافسة': 'tender_name',
    'رقم المنافسة': 'tender_number',
    'الرقم المرجعي': 'reference_number',
    'الغرض من المنافسة': 'tender_purpose',
    'قيمة وثائق المنافسة': 'tender_document_value',
    'حالة المنافسة': 'tender_status',
    'مدة العقد': 'contract_duration',
    'هل التأمين من متطلبات المنافسة': 'insurance_required',
    'نوع المنافسة': 'tender_type',
    'الجهة الحكوميه': 'government_entity',
    'الوقت المتبقى': 'time_remaining',
    'طريقة تقديم العروض': 'submission_method',
    'مطلوب ضمان الإبتدائي': 'initial_guarantee_required',
    'الضمان النهائي': 'final_guarantee',
    'آخر موعد لإستلام الإستفسارات': 'inquiry_deadline',
    'آخر موعد لتقديم العروض': 'submission_deadline',
    'تاريخ فتح العروض': 'opening_date',
    'تاريخ فحص العروض': 'evaluation_date',
    'فترة التوقف': 'suspension_period',
    'التاريخ المتوقع للترسية': 'expected_award_date',
    'تاريخ بدء الأعمال / الخدمات': 'work_start_date',
    'بداية إرسال الأسئلة و الاستفسارات': 'question_start_date',
    'اقصى مدة للاجابة على الاستفسارات': 'max_answer_days',
    'مكان فتح العرض': 'opening_location',
    'مجال التصنيف': 'classification_field',
    'مكان التنفيذ': 'execution_location',
    'التفاصيل': 'details',
    'نشاط المنافسة': 'competition_activity',
    'تشمل المنافسة على بنود توريد': 'includes_supply_items',
    'أعمال الصيانة والتشغيل': 'maintenance_works',
    'عنوان الضمان الإبتدائى': 'initial_guarantee_address',
    'أعمال  الإنشاء': 'construction_works',
    'الحزمة': 'package',
    'نوع الاتفاقية': 'agreement_type',
    'مدة الاتفاقية': 'agreement_duration',
    'عدد ايام استلام الاستفسارات': 'inquiry_days',
    'عدد أيام استلام العرض': 'submission_days',
    'قائمة الموردين المتقدمين': 'suppliers_applied',
    'قائمة الموردين المرسى عليهم  -  ( ترسية كاملة )': 'suppliers_awarded',
    'الدول': 'countries'
}, inplace=True)

In [37]:
def clean_budget(x):
    try:
        if pd.isnull(x):
            return None
        x = str(x).replace(',', '').replace(' ', '')
        if x.lower() in ['مجانا', 'free']:
            return 0
        return float(x)
    except:
        return None

df['tender_document_value'] = df['tender_document_value'].apply(clean_budget)


In [38]:
yes_no_cols = ['insurance_required', 'initial_guarantee_required']
for col in yes_no_cols:
    df[col] = df[col].map({'نعم': True, 'لا': False, True: True, False: False})


In [39]:
date_cols = [
    'inquiry_deadline', 'submission_deadline', 'opening_date', 'evaluation_date',
    'expected_award_date', 'work_start_date', 'question_start_date'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

In [40]:
df['source_file'] = 'nabahan_full_details.csv'
df['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')


In [41]:
df.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\core\nabahan_full_details_clean.csv", index=False)
print("✅ Preprocessing complete. Cleaned file saved to processed/core/tenders_clean.csv")

✅ Preprocessing complete. Cleaned file saved to processed/core/tenders_clean.csv


# Preprocessing: Future Projects Dataset

## Dataset Overview
This dataset contains future project plans from government entities. It includes 37,764 entries with 12 columns covering:

- Project identifiers (`id`, `project_name`)
- Responsible government entities (`government_entity`)
- Time periods (`quarter`, `year`)
- Locations (`execution_location`)
- Project details (`project_nature`, `project_description`, `project_status`)
- Expected durations in days, months, and years (`expected_duration_days`, `expected_duration_months`, `expected_duration_years`)

All columns are mostly complete, with very few missing values (`expected_duration_years` has a few missing).

## Preprocessing Steps

1. **Rename Columns**
   - Converted all columns to **English snake_case** for consistency and analysis.

2. **Normalize Numeric Fields**
   - Duration in years is converted to float (`NaN` if missing).

3. **Keep Text Fields**
   - All text fields are preserved as strings.

4. **Add Metadata**
   - `source_file` = `"future_projects.csv"`  
   - `ingestion_date` = current date

5. **Keep All Columns**
   - No column is dropped. Even if sparse, all are preserved for future use.

6. **Save Cleaned Dataset**
   - Cleaned data saved to: `processed/core/future_projects_clean.csv`
   - Ready for merging with other datasets like `tenders_clean.csv`.

---

> This preprocessing ensures all real data is preserved and ready for merging into a **unified tenders/projects database** for further analysis and NLP → SQL agent queries.


In [46]:
df_fp = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\core\future_projects.csv")
df_fp.head()

,#,إسم المشروع,الجهات,الربع السنوي,السنه,مكان التنفيذ,طبيعة المشروع,وصف المشروع,الحالة,مده التنفيذ المتوقعه (أيام),مده التنفيذ المتوقعه (شهور),مده التنفيذ المتوقعه (سنين)
0,1,تأمين أجهزة حاسب آلي وطابعات,أمارة منطقة المدينة المنورة,الربع الأول,2020,داخل المملكة,توريد أدهزة حاسب آلي وطابعات,توريد أدهزة حاسب آلي وطابعات,معتمد,31,1,0.0
1,2,تأمين مستلزمات الضيافة,أمارة منطقة المدينة المنورة,الربع الأول,2020,داخل المملكة,تأمين مستلزمات الضيافة,تأمين مستلزمات الضيافة,معتمد,31,2,0.0
2,3,توريد مستلزمات قرطاسية,أمارة منطقة المدينة المنورة,الربع الأول,2020,داخل المملكة,مستلزمات قرطاسية,مستلزمات قرطاسية,معتمد,31,1,0.0
3,4,توريد سيارات,أمارة منطقة المدينة المنورة,الربع الأول,2020,داخل المملكة,توريد سيارات,توريد سيارات,معتمد,31,2,0.0
4,5,تأمين محروقات,أمارة منطقة المدينة المنورة,الربع الأول,2020,داخل المملكة,تأمين محروقات,تأمين محروقات,معتمد,31,12,1.0


In [47]:
df_fp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37764 entries, 0 to 37763
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   #                            37764 non-null  int64  
 1   إسم المشروع                  37764 non-null  object 
 2   الجهات                       37764 non-null  object 
 3   الربع السنوي                 37764 non-null  object 
 4   السنه                        37764 non-null  int64  
 5   مكان التنفيذ                 37764 non-null  object 
 6   طبيعة المشروع                37764 non-null  object 
 7   وصف المشروع                  37764 non-null  object 
 8   الحالة                       37764 non-null  object 
 9   مده التنفيذ المتوقعه (أيام)  37764 non-null  int64  
 10  مده التنفيذ المتوقعه (شهور)  37764 non-null  int64  
 11  مده التنفيذ المتوقعه (سنين)  37758 non-null  float64
dtypes: float64(1), int64(4), object(7)
memory usage: 3.5+ MB


In [48]:
df_fp.columns

Index(['#', 'إسم المشروع', 'الجهات', 'الربع السنوي', 'السنه', 'مكان التنفيذ',
       'طبيعة المشروع', 'وصف المشروع', 'الحالة', 'مده التنفيذ المتوقعه (أيام)',
       'مده التنفيذ المتوقعه (شهور)', 'مده التنفيذ المتوقعه (سنين)'],
      dtype='object')

In [49]:
df_fp.rename(columns={
    '#': 'id',
    'إسم المشروع': 'project_name',
    'الجهات': 'government_entity',
    'الربع السنوي': 'quarter',
    'السنه': 'year',
    'مكان التنفيذ': 'execution_location',
    'طبيعة المشروع': 'project_nature',
    'وصف المشروع': 'project_description',
    'الحالة': 'project_status',
    'مده التنفيذ المتوقعه (أيام)': 'expected_duration_days',
    'مده التنفيذ المتوقعه (شهور)': 'expected_duration_months',
    'مده التنفيذ المتوقعه (سنين)': 'expected_duration_years'
}, inplace=True)

In [50]:
df_fp['expected_duration_years'] = pd.to_numeric(df_fp['expected_duration_years'], errors='coerce')

In [51]:
text_cols = ['project_name', 'government_entity', 'quarter', 'execution_location', 
             'project_nature', 'project_description', 'project_status']
for col in text_cols:
    df_fp[col] = df_fp[col].astype(str)

In [52]:
df_fp['source_file'] = 'future_projects.csv'
df_fp['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [53]:
df_fp.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\core\future_projects_clean.csv", index=False)
print("✅ Preprocessing complete. Cleaned file saved to processed/core/future_projects_clean.csv")

✅ Preprocessing complete. Cleaned file saved to processed/core/future_projects_clean.csv


# Government Entity Lookup Preprocessing

## Overview
Contains 1,723 government entities. Used to standardize and join with main datasets.

## Steps
- **Rename column:** `الجهة الحكوميه` → `government_entity`  
- **Clean text:** remove spaces  
- **Remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/government_entity_clean.csv`


In [ ]:
df_ge = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\government_entity_clean.csv")
df_ge.head()

,الجهة الحكوميه
0,الديوان الملكي
1,المركز الوطني للوثائق والمحفوظات
2,الأمانة العامة لمجلس المحميات الملكية
3,مكتب شؤون المهمات والمبادرات
4,المجلس الاقتصادي الأعلى


In [58]:
df_ge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1723 entries, 0 to 1722
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   الجهة الحكوميه  1723 non-null   object
dtypes: object(1)
memory usage: 13.6+ KB


In [59]:
df_ge.rename(columns={'الجهة الحكوميه': 'government_entity'}, inplace=True)

In [60]:
df_ge['government_entity'] = df_ge['government_entity'].str.strip()


In [61]:
df_ge.drop_duplicates(inplace=True)

In [62]:
df_ge['source_file'] = 'Government Entity.csv'
df_ge['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [63]:
df_ge.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\government_entity_clean.csv", index=False)
print("✅ Government Entity lookup cleaned!")

✅ Government Entity lookup cleaned!


# Primary Activity Lookup Preprocessing

## Overview
Contains 19 primary activities. Used to standardize categories in main datasets.

## Steps
- **Rename columns:** `id` → `activity_id`, `النشاط الأساسي` → `primary_activity`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/primary_activity_clean.csv`


In [72]:
df_pa = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\Primary Activity.csv")
df_pa.head()

,id,النشاط الأساسي
0,1',التجارة
1,2',المقاولات
2,3',التشغيل والصيانة والنظافة للمنشآت
3,4',العقارات والأراضي
4,5',الصناعة والتعدين والتدوير


In [73]:
df_pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                19 non-null     object
 1    النشاط الأساسي   19 non-null     object
dtypes: object(2)
memory usage: 436.0+ bytes


In [74]:
df_pa.columns = df_pa.columns.str.strip()

In [75]:
df_pa.rename(columns={'id': 'activity_id', 'النشاط الأساسي': 'primary_activity'}, inplace=True)


In [77]:
df_pa['primary_activity'] = df_pa['primary_activity'].str.strip()

In [78]:
df_pa.drop_duplicates(inplace=True)

In [79]:
df_pa['source_file'] = 'Primary Activity.csv'
df_pa['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [80]:
df_pa.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\primary_activity_clean.csv", index=False)
print("✅ Primary Activity lookup cleaned!")

✅ Primary Activity lookup cleaned!


# Regions Lookup Preprocessing

## Overview
Contains 13 regions. Used to standardize locations in main datasets.

## Steps
- **Rename column:** `المناطق` → `region_name`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/regions_clean.csv`


In [81]:
df_r = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\regions.csv")
df_r.head()

,المناطق
0,منطقة الرياض
1,منطقة مكة المكرمة
2,منطقة المدينة المنورة
3,منطقة القصيم
4,المنطقة الشرقية


In [82]:
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0    المناطق  13 non-null     object
dtypes: object(1)
memory usage: 236.0+ bytes


In [83]:
df_r.columns = df_r.columns.str.strip()

In [ ]:
df_r.rename(columns={'المناطق': 'region_name'}, inplace=True)

In [85]:
df_r['region_name'] = df_r['region_name'].str.strip()

In [86]:
df_r.drop_duplicates(inplace=True)

In [87]:
df_r['source_file'] = 'regions.csv'
df_r['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [88]:
df_r.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\regions_clean.csv", index=False)
print("✅ Regions lookup cleaned!")

✅ Regions lookup cleaned!


# Secondary Activity Lookup Preprocessing

## Overview
Contains 109 sub-activities linked to primary activities.  
Used to standardize categories in main datasets.

## Steps
- **Rename columns:** `Parent-ID` → `primary_activity_id`, `النشاط الفرعي` → `secondary_activity`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/secondary_activity_clean.csv`


In [89]:
df_sa = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\Secondary Activity.csv")
df_sa.head()

,Parent-ID,النشاط الفرعي
0,1',تجارة المواد الغذائية
1,1',تجارة الحاصلات الزراعية
2,1',تجارة المواشي والدواجن والأسماك ومنتجاتها
3,1',تجارة الملابس والأقمشة والعطور والساعات وأدوات...
4,1',تجارة الكماليات


In [90]:
df_sa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Parent-ID      109 non-null    object
 1   النشاط الفرعي  109 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [91]:
df_sa.columns = df_sa.columns.str.strip()

In [92]:
df_sa.rename(columns={'Parent-ID': 'primary_activity_id', 'النشاط الفرعي': 'secondary_activity'}, inplace=True)

In [93]:
df_sa['secondary_activity'] = df_sa['secondary_activity'].str.strip()

In [94]:
df_sa.drop_duplicates(inplace=True)

In [95]:
df_sa['source_file'] = 'Secondary Activity.csv'
df_sa['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [96]:
df_sa.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\secondary_activity_clean.csv", index=False)
print("✅ Secondary Activity lookup cleaned!")

✅ Secondary Activity lookup cleaned!


# Tender Document Price Range Lookup Preprocessing

## Overview
Contains 7 ranges for tender document prices.  
Used to standardize document price values in main datasets.

## Steps
- **Rename column:** `مدى سعر كراسة الشروط` → `tender_doc_price_range`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/tender_doc_price_range_clean.csv`


In [98]:
df_tdpr = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\Tender Document Price Range.csv")
df_tdpr.head()

,مدى سعر كراسة الشروط
0,مجانا
1,"1 - 1,000"
2,"1,001 - 10,000"
3,"10,001 - 20,000"
4,"20,001 - 40,000"


In [99]:
df_tdpr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   مدى سعر كراسة الشروط  7 non-null      object
dtypes: object(1)
memory usage: 188.0+ bytes


In [100]:
df_tdpr.columns = df_tdpr.columns.str.strip()

In [101]:
df_tdpr.rename(columns={'مدى سعر كراسة الشروط': 'tender_doc_price_range'}, inplace=True)

In [102]:
df_tdpr['tender_doc_price_range'] = df_tdpr['tender_doc_price_range'].str.strip()

In [103]:
df_tdpr.drop_duplicates(inplace=True)

In [104]:
df_tdpr['source_file'] = 'Tender Document Price Range.csv'
df_tdpr['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [105]:
df_tdpr.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\tender_doc_price_range_clean.csv", index=False)
print("✅ Tender Document Price Range lookup cleaned!")

✅ Tender Document Price Range lookup cleaned!


# Tender Publication Date Lookup Preprocessing

## Overview
Contains 5 labels for tender publication dates (e.g., "since 2 days").  
Used to standardize publication date values in main datasets.

## Steps
- **Rename column:** `تاريخ نشر المنافسة` → `tender_publication_date`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/tender_publication_date_clean.csv`


In [106]:
df_tpd = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\Tender Publication Date.csv")
df_tpd.head()

,تاريخ نشر المنافسة
0,في أى وقت
1,منذ يومين
2,منذ أسبوع
3,منذ شهر
4,منذ 3 شهور


In [107]:
df_tpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   تاريخ نشر المنافسة  5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [108]:
df_tpd.columns = df_tpd.columns.str.strip()

In [109]:
df_tpd.rename(columns={'تاريخ نشر المنافسة': 'tender_publication_date'}, inplace=True)

In [110]:
df_tpd['tender_publication_date'] = df_tpd['tender_publication_date'].str.strip()

In [111]:
df_tpd.drop_duplicates(inplace=True)

In [112]:
df_tpd['source_file'] = 'Tender Publication Date.csv'
df_tpd['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [113]:
df_tpd.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\tender_publication_date_clean.csv", index=False)
print("✅ Tender Publication Date lookup cleaned!")

✅ Tender Publication Date lookup cleaned!


# Tender Statuses Lookup Preprocessing

## Overview
Contains 7 main and sub tender statuses.  
Used to standardize tender status values in main datasets.

## Steps
- **Rename columns:** `حالة المنافسة الرئيسية` → `main_tender_status`, `حالة المنافسة الفرعية` → `sub_tender_status`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/tender_statuses_clean.csv`


In [116]:
df_ts = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\Tender_Statuses.csv")
df_ts.head()

,حالة المنافسة الرئيسية,حالة المنافسة الفرعية
0,الكل,الكل
1,المنافسات النشطة,تقديم العروض
2,المنافسات المنتهية,الكل
3,المنافسات المنتهية,مرحلة فتح العروض
4,المنافسات المنتهية,مرحلة فحص العروض


In [117]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   حالة المنافسة الرئيسية  7 non-null      object
 1   حالة المنافسة الفرعية   7 non-null      object
dtypes: object(2)
memory usage: 244.0+ bytes


In [118]:
df_ts.columns = df_ts.columns.str.strip()


In [119]:
df_ts.rename(columns={
    'حالة المنافسة الرئيسية': 'main_tender_status',
    'حالة المنافسة الفرعية': 'sub_tender_status'
}, inplace=True)

In [120]:
df_ts['main_tender_status'] = df_ts['main_tender_status'].str.strip()
df_ts['sub_tender_status'] = df_ts['sub_tender_status'].str.strip()

In [121]:
df_ts.drop_duplicates(inplace=True)

In [122]:
df_ts['source_file'] = 'Tender_Statuses.csv'
df_ts['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [123]:
df_ts.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\tender_statuses_clean.csv", index=False)
print("✅ Tender Statuses lookup cleaned!")

✅ Tender Statuses lookup cleaned!


# Tender Types Lookup Preprocessing

## Overview
Contains 13 tender types.  
Used to standardize tender type values in main datasets.

## Steps
- **Rename column:** `نوع المنافسة` → `tender_type`  
- **Clean text** and **remove duplicates**  
- **Add metadata:** `source_file` and `ingestion_date`  
- **Save cleaned file:** `Data/processed/lookup/tender_types_clean.csv`


In [126]:
df_tt = pd.read_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\raw\lookup\tender_types.csv")
df_tt.head()

,نوع المنافسة
0,منافسة عامة
1,شراء مباشر
2,منافسة محدودة
3,المزايدة العكسية الالكترونية
4,المنافسة على مرحلتين (المرحلة الاولى)


In [127]:
df_tt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   نوع المنافسة  13 non-null     object
dtypes: object(1)
memory usage: 236.0+ bytes


In [128]:
df_tt.columns = df_tt.columns.str.strip()

In [129]:
df_tt.rename(columns={'نوع المنافسة': 'tender_type'}, inplace=True)


In [130]:
df_tt['tender_type'] = df_tt['tender_type'].str.strip()


In [131]:
df_tt.drop_duplicates(inplace=True)


In [132]:
df_tt['source_file'] = 'tender_types.csv'
df_tt['ingestion_date'] = datetime.today().strftime('%Y-%m-%d')

In [ ]:
df_tt.to_csv(r"C:\Users\Rubah\Downloads\Final_Project\Data\processed\lookup\tender_types_clean.csv", index=False)
print("✅ Tender Types lookup cleaned!")

✅ Tender Types lookup cleaned!


: 